# A3: ReactionNetworks demo

# Introduction

This notebook demonstrates the functionality of the ReactionNetworks module. Classes included here are ReactionNetwork, C13ReactionNetwork and 2SReactionNetwork, which hold stochiometric networks, C13 transition networks and mixed stochiometric 13C transitions networks respectively.  

# Setup

First, we need to set the path and environment variable properly:

In [1]:
quantmodelDir = '/users/hgmartin/libraries/quantmodel'

This is the only place where the jQMM library path needs to be set.

In [2]:
%matplotlib inline

import sys, os
pythonPath = quantmodelDir+"/code/core"
if pythonPath not in sys.path:
    sys.path.append(pythonPath)
os.environ["QUANTMODELPATH"] = quantmodelDir

Then import the needed classes:

In [3]:
import ReactionNetworks as RN
import core, enhancedLists
import numpy

and get into a test directory:

In [4]:
cd ~/tests

/mnt/nfs.jbei/nfs/users/hgmartin/tests


# Classes description

## ReactionNetwork class

The ReactionNetwork class holds reaction networks with only stochiometric information, such as used for FBA. Let's create a reaction network from scratch by creating a few reactions (see core and enhancedLists module notebook for more details):

In [5]:
GLCpts = core.Reaction.from_string('GLCpts: glc_dsh_D_e + pep_c --> g6p_c + pyr_c')
PGI    = core.Reaction.from_string('PGI: g6p_c <==> f6p_c')
PFK    = core.Reaction.from_string('PFK: atp_c + f6p_c --> adp_c + fdp_c + h_c')

In [6]:
print GLCpts.stoichLine()
print PGI.stoichLine()
print PFK.stoichLine()

GLCpts : glc_dsh_D_e + pep_c --> g6p_c + pyr_c
PGI : g6p_c <==> f6p_c
PFK : atp_c + f6p_c --> adp_c + fdp_c + h_c


Once we have the reactions, we create a reactionList instance:

In [7]:
reactionList = enhancedLists.ReactionList([GLCpts,PGI,PFK])

And, from there, we create a list of metabolites with getMetList and use test as name of the reaction network and include no notes ({}):

In [8]:
ReactionNetwork = RN.ReactionNetwork(('test',{},reactionList.getMetList(),reactionList))

In [9]:
print ReactionNetwork.reactionList

GLCpts
PFK
PGI


In [10]:
print ReactionNetwork.metList

adp_c
atp_c
f6p_c
f6p_c
fdp_c
g6p_c
g6p_c
glc_dsh_D_e
h_c
pep_c
pyr_c


The **addReaction** method adds a reaction to the network:

In [11]:
ReactionNetwork.addReaction('F6PA: f6p_c <==> dha_c + g3p_c')

True

In [12]:
print ReactionNetwork.reactionList

F6PA
GLCpts
PFK
PGI


In [13]:
print ReactionNetwork.metList

adp_c
atp_c
dha_c
f6p_c
f6p_c
fdp_c
g3p_c
g6p_c
g6p_c
glc_dsh_D_e
h_c
pep_c
pyr_c


and **deleteReaction** eliminates a reaction:

In [14]:
ReactionNetwork.deleteReaction('F6PA')

In [15]:
print ReactionNetwork.reactionList

GLCpts
PFK
PGI


**changeFluxBounds** changes upper and lower bounds for a desired reaction:

In [16]:
ReactionNetwork.changeFluxBounds('GLCpts',3)
ReactionNetwork.changeFluxBounds('PGI',0)
ReactionNetwork.changeFluxBounds('PFK',(0.1,0.5))


In [17]:
rxnDict = ReactionNetwork.reactionList.getReactionDictionary()

In [18]:
print rxnDict['PFK'].fluxBounds.net

[0.1:0.3:0.5]


(First number is lower bound, last number is upper bound)

Flux bounds can also be loaded from a file using **loadFluxBounds**:

In [19]:
lines= [
    'GLCpts:  0.3 [==] 0.45',
    'PGI:  0.8 [==] 0.95',
    'PFK:  0.0 [==] 1000',    
]

In [20]:
ReactionNetwork.loadFluxBounds(('testFile.txt','\n'.join(lines)))

In [21]:
print rxnDict['GLCpts'].fluxBounds.net
print rxnDict['PGI'].fluxBounds.net

[0.3:0.3:0.45]
[0.8:0.8:0.95]


**fixFluxes** fixes the value of lower and upper found to the current flux (+/- some wiggle room):

In [22]:
rxnDict['PGI'].flux = rxnDict['PGI'].fluxBounds

In [23]:
ReactionNetwork.fixFluxes()

In [24]:
print rxnDict['PGI'].fluxBounds.net

[0.792:0.8:0.808]


This is useful when you want to constrains fluxes to a measured value, e.g. for finding the OF corresponding to that measured value.

and **capFluxBounds** caps the maximum bounds for each reaction:

In [25]:
ReactionNetwork.capFluxBounds(10)

In [26]:
print rxnDict['PFK'].fluxBounds

Forward: NA
Backward: NA
Net: [0:0:10]
Exchange: NA



**getStoichMetRxnFBAFiles** produces the GAMS files for running FBA:

In [27]:
ReactionNetwork.getStoichMetRxnFBAFiles()

[('Sbig.txt',
  "Sbig('adp_c','PFK')=1;\nSbig('atp_c','PFK')=-1;\nSbig('f6p_c','PFK')=-1;\nSbig('f6p_c','PGI')=1;\nSbig('fdp_c','PFK')=1;\nSbig('g6p_c','GLCpts')=1;\nSbig('g6p_c','PGI')=-1;\nSbig('glc_dsh_D_e','GLCpts')=-1;\nSbig('h_c','PFK')=1;\nSbig('pep_c','GLCpts')=-1;\nSbig('pyr_c','GLCpts')=1;\n"),
 ('hybridmets.txt',
  "/ \n'adp_c'\n'atp_c'\n'f6p_c'\n'fdp_c'\n'g6p_c'\n'glc_dsh_D_e'\n'h_c'\n'pep_c'\n'pyr_c'\n/ \n"),
 ('hybridreacs.txt', "/ \n'GLCpts'\n'PFK'\n'PGI'\n/ \n"),
 ('jGEMsugg.txt', '/ \n/ \n'),
 ('vFBASUGG.txt', ''),
 ('ubvec.txt',
  "ubvec('GLCpts')=0.45;\nubvec('PFK')=10;\nubvec('PGI')=0.808;\n"),
 ('lbvec.txt',
  "lbvec('GLCpts')=0.3;\nlbvec('PFK')=0.0;\nlbvec('PGI')=0.792;\n"),
 ('cvec.txt', ''),
 ('bvec.txt',
  "bvec('adp_c')=0;\nbvec('atp_c')=0;\nbvec('f6p_c')=0;\nbvec('fdp_c')=0;\nbvec('g6p_c')=0;\nbvec('glc_dsh_D_e')=0;\nbvec('h_c')=0;\nbvec('pep_c')=0;\nbvec('pyr_c')=0;\n")]

as does **getStoichMetRxnFVAFiles** for Flux Variability Analysis (FVA).

The method **write** writes the reaction format in SBML format for the given filename (or to a string if 'toString' is used as arugment):

In [28]:
print ReactionNetwork.write('toString')

<?xml version="1.0" encoding="UTF-8"?>
<sbml xmlns="http://www.sbml.org/sbml/level2" xmlns:html="http://www.w3.org/1999/xhtml" level="2" version="1">
  <model id="test" name="test">
    <notes>
      <body xmlns="http://www.w3.org/1999/xhtml"/>
    </notes>
    <listOfUnitDefinitions>
      <unitDefinition id="mmol_per_gDW_per_hr">
        <listOfUnits>
          <unit kind="mole" scale="-3"/>
          <unit kind="gram" exponent="-1"/>
          <unit kind="second" exponent="-1" multiplier="0.00027777"/>
        </listOfUnits>
      </unitDefinition>
    </listOfUnitDefinitions>
    <listOfCompartments>
      <compartment id="c" name="cytosol"/>
      <compartment id="e" name="extracellular space"/>
    </listOfCompartments>
    <listOfSpecies>
      <species id="M_adp_c" name="M_adp_c" compartment="c">
        <notes>
          <body xmlns="http://www.w3.org/1999/xhtml">
            <p>CARBON NUMBER: 0</p>
          </body>
        </notes>
      </species>
      <species id="M_atp_c

## C13ReactionNetwork class

The C13ReactionNetwork class holds reaction networks with only carbon transition information, such as used for 13C MFA. Let's upload a C13ReactionNetwork for a TCA cycle mock-up:

In [29]:
qmodeldir         = os.environ['QUANTMODELPATH']    
dirDATA           = qmodeldir+'/data/tests/TCAtoy/' 


REACTIONSfilename   = dirDATA+'REACTIONStca.txt' 
FEEDfilename        = dirDATA+'FEEDtca.txt'
CEMSfilename        = dirDATA+'GCMStca.txt'
CEMSSTDfilename     = dirDATA+'GCMSerrtca.txt'
FLUXESFreefilename  = dirDATA+'FLUXtca.txt'

**addLabeling** and **addFeed** are used to load the labeling data and the feed labeling information""

In [30]:
atomTransitions = enhancedLists.AtomTransitionList(REACTIONSfilename)
ReacNet = ReactionNetworks.C13ReactionNetwork(atomTransitions.getReactionNetwork('E. coli wt5h 13C MFA'))

ReacNet.addLabeling(CEMSfilename,'LCMSLabelData',CEMSSTDfilename,minSTD=0.001)
ReacNet.addFeed(FEEDfilename)
ReacNet.loadFluxBounds(FLUXESFreefilename)

Here is the labeling (Mass Distribution Vector, or MDV) for glutamate:

In [31]:
ReacNet.notes['LCMSLabelData']['Glu'].mdv

array([ 0.346,  0.269,  0.27 ,  0.081,  0.028,  0.004])

In [32]:
ReacNet.notes['LCMSLabelData']['Glu'].std

array([ 0.02,  0.02,  0.02,  0.02,  0.02,  0.02])

One can use **randomizeLabeling** to randomize this labeling within the standard deviations:

In [33]:
ReacNet.randomizeLabeling()
print ReacNet.notes['LCMSLabelData']['Glu'].mdv

[ 0.35771396  0.25660007  0.28460434  0.07200119  0.0238263   0.00042019]


In [34]:
ReacNet.randomizeLabeling()
print ReacNet.notes['LCMSLabelData']['Glu'].mdv

[ 0.35442818  0.27511906  0.28995896  0.06763062  0.01730401  0.00317596]


**fragDict** produces a dictionary of fragments to be fit:

In [35]:
fragDict = ReacNet.fragDict()

In [36]:
print ReacNet.fragDict()

{'Glu': <labeling.LCMSfragment instance at 0x7f730324bcf8>}


In [37]:
print fragDict['Glu'].mdv
print fragDict['Glu'].std

[ 0.35442818  0.27511906  0.28995896  0.06763062  0.01730401  0.00317596]
[ 0.02  0.02  0.02  0.02  0.02  0.02]


**getLabelingDataFiles** produces the gams files with the labeling information:

In [38]:
ReacNet.getLabelingDataFiles()

[('GCMSout.txt',
  'Table labelexp(frag,n)\n\t0\t1\t2\t3\t4\t5\t6\t7\t8\t9\t10\t11\t12\nGlu\t0.35443\t0.27512\t0.28996\t0.06763\t0.01730\t0.00318\t0.00000\t0.00000\t0.00000\t0.00000\t0.00000\t0.00000\t0.00000\n;'),
 ('GCMSerr.txt',
  'Table labelstd(frag,n)\n\t0\t1\t2\t3\t4\t5\t6\t7\t8\t9\t10\t11\t12\nGlu\t0.02000\t0.02000\t0.02000\t0.02000\t0.02000\t0.02000\t0.00000\t0.00000\t0.00000\t0.00000\t0.00000\t0.00000\t0.00000\n;'),
 ('GCMSnorm.txt',
  'Table labelnorm(frag,n)\n\t0\t1\t2\t3\t4\t5\t6\t7\t8\t9\t10\t11\t12\nGlu\t6.00000\t6.00000\t6.00000\t6.00000\t6.00000\t6.00000\t0.00000\t0.00000\t0.00000\t0.00000\t0.00000\t0.00000\t0.00000\n;')]

**getFragmentInfoFiles** produces the gams files with the fragment information (raw indicates that no derivatization correction is needed):

In [39]:
ReacNet.getFragmentInfoFiles('raw')

[('aacorr.txt', "aacorr('Glu','glu_L_c_1_2_3_4_5')=1;\n"),
 ('mset.txt',
  "/ \n'0'\n'1'\n'2'\n'3'\n'4'\n'5'\n'6'\n'7'\n'8'\n'9'\n'10'\n'11'\n'12'\n/ \n"),
 ('nset.txt',
  "/ \n'0'\n'1'\n'2'\n'3'\n'4'\n'5'\n'6'\n'7'\n'8'\n'9'\n'10'\n'11'\n'12'\n/ \n"),
 ('frag_carbon_numbers.txt', "ncarbonsf('Glu')=5;\n"),
 ('fragmentsMS.txt', "/ \n'Glu'\n/ \n"),
 ('fitFrags.txt', "/ \n'Glu'\n/ \n"),
 ('gamma_mat.txt',
  "gamma('Glu','0','0')=0.987985018979;\ngamma('Glu','0','1')=0.0;\ngamma('Glu','0','10')=0.0;\ngamma('Glu','0','11')=0.0;\ngamma('Glu','0','12')=0.0;\ngamma('Glu','0','2')=0.0;\ngamma('Glu','0','3')=0.0;\ngamma('Glu','0','4')=0.0;\ngamma('Glu','0','5')=0.0;\ngamma('Glu','0','6')=0.0;\ngamma('Glu','0','7')=0.0;\ngamma('Glu','0','8')=0.0;\ngamma('Glu','0','9')=0.0;\ngamma('Glu','1','0')=0.00583741423328;\ngamma('Glu','1','1')=0.987985018979;\ngamma('Glu','1','10')=0.0;\ngamma('Glu','1','11')=0.0;\ngamma('Glu','1','12')=0.0;\ngamma('Glu','1','2')=0.0;\ngamma('Glu','1','3')=0.0;\ngamma('Glu

**getSourceLabelFile** produces the file with the source labeling information:

In [40]:
ReacNet.getSourceLabelFile()

[('Source_Labeling.txt',
  "f.fx('accoa_c_1_2','0')=0.5;\nf.fx('accoa_c_1_2','1')=0.25;\nf.fx('accoa_c_1_2','2')=0.25;\n*\nf.fx('accoa_c_2','0')=0.5;\nf.fx('accoa_c_2','1')=0.5;\n*\nf.fx('asp_c_1_2_3','0')=1.0;\nf.fx('asp_c_1_2_3','1')=0.0;\nf.fx('asp_c_1_2_3','2')=0.0;\nf.fx('asp_c_1_2_3','3')=0.0;\n*\nf.fx('asp_c_2','0')=1.0;\nf.fx('asp_c_2','1')=0.0;\n*\nf.fx('asp_c_2_3','0')=1.0;\nf.fx('asp_c_2_3','1')=0.0;\nf.fx('asp_c_2_3','2')=0.0;\n*\nf.fx('asp_c_2_3_4','0')=1.0;\nf.fx('asp_c_2_3_4','1')=0.0;\nf.fx('asp_c_2_3_4','2')=0.0;\nf.fx('asp_c_2_3_4','3')=0.0;\n*\nf.fx('asp_c_3','0')=1.0;\nf.fx('asp_c_3','1')=0.0;\n*\n*\n")]

**getStoichMetRxn13CFiles** produces the files with the metabolites, reactions and stoichiometric information for 13C MFA:

In [41]:
ReacNet.getStoichMetRxn13CFiles()

[('StoichNetwork.txt',
  "S('accoa_c','r1')=-1;\nS('akg_c','r2')=1;\nS('akg_c','r3')=-1;\nS('akg_c','r4')=-1;\nS('asp_c','r8')=-1;\nS('cit_c','r1')=1;\nS('cit_c','r2')=-1;\nS('co2_c','co2Out')=-1;\nS('co2_c','r2')=1;\nS('co2_c','r4')=1;\nS('co2ext_c','co2Out')=1;\nS('fum_c','r5')=1;\nS('fum_c','r6')=-1;\nS('fum_c','r7')=1;\nS('glu_L_c','r3')=1;\nS('oac_c','r1')=-1;\nS('oac_c','r6')=1;\nS('oac_c','r7')=-1;\nS('oac_c','r8')=1;\nS('suc_c','r4')=1;\nS('suc_c','r5')=-1;\n"),
 ('mets.txt',
  "/ \n'accoa_c'\n'akg_c'\n'asp_c'\n'cit_c'\n'co2_c'\n'co2ext_c'\n'fum_c'\n'glu_L_c'\n'oac_c'\n'suc_c'\n/ \n"),
 ('rxns.txt',
  "/ \n'co2Out'\n'r1'\n'r2'\n'r3'\n'r4'\n'r5'\n'r6'\n'r7'\n'r8'\n/ \n"),
 ('ExchaMets.txt', "/ \n'accoa_c'\n'asp_c'\n'co2ext_c'\n'glu_L_c'\n/ \n"),
 ('jmeas.txt', '/ \n/ \n'),
 ('vUPPER.txt', ''),
 ('vLOWER.txt', ''),
 ('ExcMets.txt', '/ \n/ \n'),
 ('SourceMets.txt', "/ \n'accoa_c'\n'asp_c'\n/ \n"),
 ('jsugg.txt', '/ \n/ \n'),
 ('vSUGG.txt', '')]

**getEMUfiles** produces the EMU information files:

In [42]:
ReacNet.getEMUfiles()

[('all_emu.txt',
  "/ \n'accoa_c_1_2'\n'accoa_c_1_2oac_c_2'\n'accoa_c_1_2oac_c_2_3_4'\n'accoa_c_2'\n'accoa_c_2oac_c_2'\n'accoa_c_2oac_c_2_3'\n'akg_c_1_2_3_4_5'\n'akg_c_2_3_4'\n'akg_c_3'\n'akg_c_3_4'\n'akg_c_3_4_5'\n'akg_c_4'\n'asp_c_1_2_3'\n'asp_c_2'\n'asp_c_2_3'\n'asp_c_2_3_4'\n'asp_c_3'\n'cit_c_1_2_3_4_5'\n'cit_c_2_3_4'\n'cit_c_3'\n'cit_c_3_4'\n'cit_c_3_4_5'\n'cit_c_4'\n'fum_c_1_2_3'\n'fum_c_2'\n'fum_c_2_3'\n'fum_c_2_3_4'\n'fum_c_3'\n'glu_L_c_1_2_3_4_5'\n'oac_c_1_2_3'\n'oac_c_2'\n'oac_c_2_3'\n'oac_c_2_3_4'\n'oac_c_3'\n'suc_c_1_2_3'\n'suc_c_2'\n'suc_c_2_3'\n'suc_c_2_3_4'\n'suc_c_3'\n/ \n"),
 ('met_emu.txt',
  "metemu('accoa_c','accoa_c_1_2')=1.0;\nmetemu('accoa_c','accoa_c_2')=1.0;\nmetemu('akg_c','akg_c_1_2_3_4_5')=1.0;\nmetemu('akg_c','akg_c_2_3_4')=1.0;\nmetemu('akg_c','akg_c_3')=1.0;\nmetemu('akg_c','akg_c_3_4')=1.0;\nmetemu('akg_c','akg_c_3_4_5')=1.0;\nmetemu('akg_c','akg_c_4')=1.0;\nmetemu('asp_c','asp_c_1_2_3')=1.0;\nmetemu('asp_c','asp_c_2')=1.0;\nmetemu('asp_c','asp_c_2_3')=1

**getAuxiliaryFiles** provides the auxiliary files (random number seed, etc):

In [43]:
ReacNet.getAuxiliaryFiles()

[('randseed.txt', 'randseed = 323334316;'),
 ('labelCompNormMin.txt', 'labelCompNormMin = 0.99;')]

## TSReactionNetwork class

The TSReactionNetwork class holds reaction networks with stochiometric information and carbon transition information for a defined core, which are typically used for 2S-13C MFA:

In [44]:
datadir           = os.environ["QUANTMODELPATH"]+'/data/tests/Toya2010/2S/wt5h/'

BASEfilename      = datadir + 'EciJR904TKs.xml'
FLUXESfilename    = datadir + 'FLUXwt5h.txt'
REACTIONSfilename = datadir + 'REACTIONSwt5h.txt'      
MSfilename        = datadir + 'GCMSwt5h.txt'
FEEDfilename      = datadir + 'FEEDwt5h.txt'
MSSTDfilename     = datadir + 'GCMSerrwt5h.txt'

# Load initial SBML file
ReacNet = RN.TSReactionNetwork(BASEfilename)
    
# Add Measured fluxes
ReacNet.loadFluxBounds(FLUXESfilename)
# Add carbon transitions
ReacNet.addTransitions(REACTIONSfilename)
# Add measured labeling information
ReacNet.addLabeling(MSfilename,'LCMSLabelData',MSSTDfilename,minSTD=0.001)
# Add feed labeling information
ReacNet.addFeed(FEEDfilename)

**addTransitions** adds the transition information to a genome-scale model:

In [45]:
rxnDict = ReacNet.reactionList.getReactionDictionary()
print rxnDict['PDH'].stoichLine()

PDH : coa_c + nad_c + pyr_c --> accoa_c + co2_c + nadh_c


In [46]:
REACTIONSfilename = datadir + 'REACTIONSwt5h.txt' 
ReacNet.addTransitions(REACTIONSfilename)

In [47]:
rxnDict = ReacNet.reactionList.getReactionDictionary()
print rxnDict['PDH'].stoichLine()
print rxnDict['PDH'].transitionLine

PDH : coa_c + nad_c + pyr_c --> accoa_c + co2_c + nadh_c
PDH	pyr_c --> co2_c + accoa_c	abc : a + bc


**getFluxRefScale** provides the flux that is used as reference scale :

In [48]:
ReacNet.getFluxRefScale(inFluxRefs=['EX_glc(e)','EX_glc_e_'])

11.7